# Puck control and possession
Expanding on Matt Cane's definition of penalty kill aggressiveness, I consider controlled exits, passing plays in the neutral zone, and cycling back out of the offesnive zone.

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
plt.rcParams['figure.dpi'] = 200
plt.rcParams['figure.facecolor'] = 'white'

## Load data

In [3]:
df = pd.read_csv('data/BDC_2024_Data_Cleaned.csv', index_col=0)
entries = pd.read_csv('data/BDC_2024_Zone_Entries.csv', index_col=0)

## Find plays

In [4]:
def play_of_interest(row):
    if (
        (row['Event'] == 'Faceoff Win') | 
        (row['Event'] == 'Penalty')
    ):
        return np.nan
    elif row['Next Event'] == 'Faceoff Win':
        # offside or icing
        if row['Event'] == 'Dump In/Out':
                return 'Failed dump in'
        # offside
        if (
            (row['Next X Coordinate'] == 80) |
            (row['Next X Coordinate'] == 120)
        ):
            return 'Failed controlled entry'
        else:
            return np.nan
    # possession maintained
    elif row['Pos Retained']:
        # puck enters offensive zone
        if (
            (row['X Coordinate'] < 125) & 
            (row['Next X Coordinate'] > 125)
        ):
            if row['Event'] == 'Dump In/Out':
                if row['X Coordinate'] > 75:
                    return 'Dump in'
                else:
                    return 'Dump out'
            elif (
                (row['Next X Coordinate'] < 130) & 
                (row['Next Event'] == 'Dump In/Out')
            ):
                return 'Dump in'
            # require entry info from given database
            elif row['Entry']:
                return 'Controlled entry'
            else:
                return np.nan
        # puck exits defensive zone
        elif (
            (row['X Coordinate'] < 75) & 
            (row['Next X Coordinate'] > 75)
        ):
            if row['Event'] == 'Dump In/Out':
                return 'Dump out'
            # TODO restrict closeness?
            else:
                return 'Controlled exit'
        # puck exits offensive zone
        elif (
            (row['X Coordinate'] > 125) & 
            (row['Next X Coordinate'] < 125)
        ):
            if row['Event'] == 'Dump In/Out':
                return 'Dump back'
            elif (
                (row['Event'] == 'Play') &
                (row['X Coordinate 2'] < 125)
            ):
                return 'Cycle back'
            else:
                return np.nan
        # puck is played within neutral zone
        elif (
            (row['X Coordinate'] > 75) & 
            (row['X Coordinate 2'] < 125)
        ):
            if row['Event'] == 'Play':
                return 'Neutral zone pass'
            if row['Event'] == 'Incomplete Play':
                return 'Failed neutral zone pass'
            else:
                return np.nan
        else:
            return np.nan
    # possession lost
    else:   
        # failed dump in
        if (
            (row['X Coordinate'] > 100) & 
            (row['X Coordinate'] < 125) & 
            (row['Next X Coordinate'] > 75) &
            (row['Event'] == 'Dump In/Out')
        ):
            return 'Failed dump in'
        # puck stays in offensive zone
        elif (
            (row['X Coordinate'] < 125) & 
            (row['Next X Coordinate'] < 75)
        ):
            if row['Event'] == 'Dump In/Out':
                if row['X Coordinate'] > 75:
                    return 'Dump in'
                else:
                    return 'Dump out'
            # puck gets taken away
            elif (
                (row['Next Event'] == 'Puck Recovery') | 
                (row['Next Event'] == 'Takeaway') |
                (row['Next Event'] == 'Penalty Taken') 
            ):
                # near the blue line
                if row['Next X Coordinate'] > 65:
                    return 'Failed controlled entry'
                # deep in the zone
                # require entry info from given database
                elif row['Entry']:
                    return 'Controlled entry'
                else:
                    return np.nan
            # entry somehow failed another way
            else:
                # pass from outside offensive zone
                if (
                    (row['Event'] == 'Play') | 
                    (row['Event'] == 'Incomplete Play')
                ):
                    # intended recipient was in offensive zone
                    if row['X Coordinate 2'] >  125:
                        return 'Failed controlled entry'
                    else:
                        return np.nan
                else:
                    return np.nan
        # puck exits defensive zone
        elif (
            (row['X Coordinate'] < 75) & 
            (row['Next X Coordinate'] < 125)
        ):
            if row['Event'] == 'Dump In/Out':
                return 'Dump out'
            # puck gets taken away
            elif (
                (row['Next Event'] == 'Puck Recovery') | 
                (row['Next Event'] == 'Takeaway')
            ):
                # near the blue line
                if row['Next X Coordinate'] > 115:
                    return 'Failed controlled exit'
                else:
                    return np.nan
            # exit somehow failed another way
            else:
                # pass from inside defensive zone
                if (
                    (row['Event'] == 'Play') | 
                    (row['Event'] == 'Incomplete Play')
                ):
                    # intended recipient was in neutral zone
                    if row['X Coordinate 2'] > 75:
                        return 'Failed controlled exit'
                    else:
                        return np.nan
                else:
                    return np.nan
        # puck exits offensive zone
        elif (
            (row['X Coordinate'] > 125) & 
            (row['Next X Coordinate'] > 75)
        ):
            if row['Event'] == 'Dump In/Out':
                return 'Dump back'
            # shot or goal led to faceoff win
            elif row['Next Event'] == 'Faceoff Win':
                return np.nan
            elif (
                (row['Event'] == 'Play') | 
                (row['Event'] == 'Incomplete Play')
            ):
                if row['X Coordinate 2'] < 125:
                    return 'Failed cycle back'
                else:
                    return np.nan
            # some puck recoveries
            else:
                return np.nan
        # puck stays in defensive zone
        elif (
            (row['X Coordinate'] < 75) & 
            (row['Next X Coordinate'] > 125)
        ):
            if row['Event'] == 'Dump In/Out':
                return 'Failed dump out'
            # puck gets taken away
            elif (
                (row['Event'] == 'Puck Recovery') | 
                (row['Event'] == 'Takeaway')
            ):
                # near the blue line
                if row['Next X Coordinate'] < 135:
                    return 'Failed controlled exit'
                else:
                    return np.nan
            else:
                return np.nan
        # puck is played within neutral zone
        elif (
            (row['X Coordinate'] > 75) & 
            (row['X Coordinate 2'] < 125)
        ):
            if row['Event'] == 'Play':
                return 'Neutral zone pass'
            if row['Event'] == 'Incomplete Play':
                return 'Failed neutral zone pass'
            else:
                return np.nan
        else:
            return np.nan

In [5]:
df['Next X Coordinate'] = df.shift(-1)['X Coordinate']
df['Next Event'] = df.shift(-1)['Event']
df['Entry'] = df.index.isin(entries.index - 1)

In [6]:
df['Play of Interest'] = df.apply(lambda row: play_of_interest(row), axis=1)

Acceptable false negatives compared to given DB for zone entries
- End of period (last play is zone entry)
- Defined as failed entry (puck taken away within 10 ft, offside, or icing)
- Defined dump out

\*one entry in the original database cannot be inferred from given data and is omitted

There are many dump ins which were not recorded as zone entries in the original database

In [7]:
df['Play of Interest'].to_csv('data/BDC_2024_Plays_of_Interest.csv')

In [8]:
df['Play of Interest'].value_counts()

Controlled exit             315
Controlled entry            314
Dump in                     248
Dump out                    192
Neutral zone pass           176
Failed dump out              67
Failed controlled entry      49
Failed neutral zone pass     46
Failed dump in               32
Failed controlled exit       23
Cycle back                    1
Failed cycle back             1
Dump back                     1
Name: Play of Interest, dtype: int64

In [9]:
pk = df.loc[df['Team Status'] == 'PK'].copy()
pp = df.loc[df['Team Status'] == 'PP'].copy()
p = df.loc[(df['Team Status'] == 'PK') | (df['Team Status'] == 'PP')]

In [10]:
pk['Play of Interest'].value_counts()

Dump out                    79
Failed dump out             28
Controlled entry            13
Dump in                      8
Neutral zone pass            5
Failed controlled entry      3
Controlled exit              2
Failed dump in               2
Failed neutral zone pass     1
Cycle back                   1
Name: Play of Interest, dtype: int64

In [11]:
pp['Play of Interest'].value_counts()

Controlled entry            58
Neutral zone pass           44
Controlled exit             43
Dump in                     22
Failed controlled entry      6
Failed neutral zone pass     5
Failed controlled exit       3
Dump out                     1
Name: Play of Interest, dtype: int64

In [12]:
p['Play of Interest'].value_counts()

Dump out                    80
Controlled entry            71
Neutral zone pass           49
Controlled exit             45
Dump in                     30
Failed dump out             28
Failed controlled entry      9
Failed neutral zone pass     6
Failed controlled exit       3
Failed dump in               2
Cycle back                   1
Name: Play of Interest, dtype: int64